In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from scipy.stats import shapiro, kstest, probplot
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from statsmodels.graphics.gofplots import qqplot


### Carga dos dados

In [ ]:
# Carregar o dataset
df_scores = pd.read_csv('./datasets/dataset.csv')

In [ ]:
df_scores.info()

In [ ]:
df_scores.head(10)

### EDA

In [ ]:
# Medidas estatísticas das variáveis
df_scores.describe()

In [ ]:
# Plot de dispersão
sns.scatterplot(data=df_scores, x='horas_estudo', y='pontuacao_teste')

In [ ]:
# Verificar outliers - box plot
sns.boxplot(data=df_scores, y='horas_estudo')

In [ ]:
# Verificar outliers - box plot
sns.boxplot(data=df_scores, y='pontuacao_teste')

In [ ]:
# Verificar correlação entre horas_estudo e pontuacao_teste - Pearson
sns.heatmap(df_scores.corr('pearson'), annot=True)

In [ ]:
# Spearman
sns.heatmap(df_scores.corr('spearman'), annot=True)

In [ ]:
# Característica do dataframe em relação à sua distribuição
sns.displot(df_scores, x='horas_estudo')

In [ ]:
sns.displot(df_scores, x='pontuacao_teste')

### Treinar modelo

In [ ]:
# Divir dados entre treianmento e teste
# Quando temos apenas uma feature, precisamos ajustar o shape (normalização)
x = df_scores['horas_estudo'].values.reshape(-1, 1)
y = df_scores['pontuacao_teste'].values.reshape(-1, 1)

x_train, x_test, y_train, y_test = train_test_split(x, y, train_size=0.7, random_state=50)

In [ ]:
# Instanciar o modelo a ser treinado
reg_model = LinearRegression()

In [ ]:
# Treinar o modelo
reg_model.fit(x_train, y_train)

In [ ]:
# Verificar a equação da reta
print("y = {:4f}x + {:4f}".format(reg_model.coef_[0][0], reg_model.intercept_[0]))

### Validação do modelo - Métricas

In [ ]:
# Predição dos valores com base no conjunto de teste
y_pred = reg_model.predict(x_test)

In [ ]:
# Calcular métrica R-squared ou Coeficiente de Determinação
# R2 - Representa a proporção na variação da variável dependente que é explicada pela variável independente
from sklearn.metrics import r2_score, mean_absolute_error, mean_squared_error, root_mean_squared_error
r2_score(y_test, y_pred)

In [ ]:
# Métrica de erro: Erro médio absoluto (MAE) - mean(y_test - y_pred)
# Fácil de interpretar, pois possui a mesma unidade da saída. Menos sensível a outliers
mean_absolute_error(y_test, y_pred)

In [ ]:
# Métrica de erro: Erro médio quadrático (MSE) - mean((y_test - y_pred)^2)
# Mais sensível a outliers e penaliza grandes errors
mean_squared_error(y_test, y_pred)

In [ ]:
# Métrica de erro: Raiz do erro médio quadrático (RMSE)
# Sensível a outliers e volta à unidade de saída. Penaliza grandes erros, ao mesmo tempo que é mais fácil de ser analisada
root_mean_squared_error(y_test, y_pred)

In [ ]:
# Análise gráfica
x_axis = range(len(y_test))
plt.figure(figsize=(10,6))
sns.scatterplot(x=x_axis, y=y_test.reshape(-1), color='blue', label='Valores reais')
sns.scatterplot(x=x_axis, y=y_pred.reshape(-1), color='red', label='Valores preditos')
plt.legend()
plt.show()

### Análise de resíduos

In [ ]:
# Calcular resíduos
residual = y_test - y_pred

In [ ]:
# Calcular os resíduos padronizados - Standardization (z score)
# z = (x - media) / desvio_padrao
from scipy.stats import zscore
std_residual = zscore(residual)

In [ ]:
# Verificar linearidade do modelo
# Se os resíduos estiverem entre -2 e +2, indica linearidade dos resíduos


# Verificar a homogeneidade das variâncias dos resíduos (homocedasticidade)
# Se os valores estiverem em torno da reta, temos homocedasticidade, caso contrário 
# Se tivermos alguma tendência ou padrão nos dados (formam um cone, funil), há heterocedasticidade

sns.scatterplot(x=y_pred.reshape(-1), y=std_residual.reshape(-1))
plt.axhline(y=0)

In [ ]:
# Avaliar se os resíduos seguem uma distribuição normal
# QQ (Quantile-Quantile) plot, que avalia se uma amostra segue uma distribuição normal
import pingouin as pg
pg.qqplot(std_residual, dist='norm', confidence=0.95)
plt.xlabel('Quantis teóricos')
plt.ylabel('Resíduos na escala padrão')

In [ ]:
# Teste de normalidade - Shapiro Wilk
# H0 - segue distribuição normal
# H1 - não segue distribuição normal
# Se o p-value > 0.05, seguimos com H0. Caso contrário, rejeitamos H0
stat_shapiro, p_value_shapiro = shapiro(residual.reshape(-1))
print("Estatísticas do teste: {}\nP-Value: {}".format(stat_shapiro, p_value_shapiro))

In [ ]:
# Teste de normalidade - Kolmogorov-Smirnov
# H0 - segue distribuição normal
# H1 - não segue distribuição normal
# Se o p-value > 0.05, seguimos com H0. Caso contrário, rejeitamos H0
stat_ks, p_value_ks = kstest(residual.reshape(-1), 'norm')
print("Estatísticas do teste: {}\nP-Value: {}".format(stat_ks, p_value_ks))

### Predições com o modelo

In [ ]:
# Se eu estudar 30.4 horas, qual a pontuação prevista pelo modelo?
reg_model.predict([[30.4]])

In [ ]:
# Se eu preciso de 600 pontos, quantas horas de estudos o modelo prevê como necessárias?
x = (600 - reg_model.intercept_[0]) / reg_model.coef_[0][0]
print(x)

### Exportar modelo

In [ ]:
import joblib
joblib.dump(reg_model, './regression_model.pkl')